# Technical Validation

In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np
import pandas as pd
from plotly.subplots import make_subplots

from htc.settings import settings
from htc.tivita.DataPath import DataPath
from htc.utils.ColorcheckerReader import ColorcheckerReader
from htc.utils.SpectrometerReader import SpectrometerReader
from htc.utils.visualization import add_figcaption, add_std_fill, colorchecker_fig_styling, compress_html

In [2]:
# Change to None for non-normalized visualization
normalization = 1

# load spectrometer measurements:
rows = []
spectrometer_cc_dir = (
    settings.data_dirs["HeiPorSPECTRAL"] / "extra_technical_validation/spectrometer/2022_09_19_colorchecker"
)
reader = SpectrometerReader(spectrometer_cc_dir)
for c, label_name in enumerate(ColorcheckerReader.label_colors_classic.keys()):
    spec = reader.read_spectrum(label_name, calibration=True, adapt_to_tivita=True, normalization=normalization)
    rows.append(["spectrometer", c + 1, label_name, spec["wavelengths"], spec["median_spectrum"], spec["std_spectrum"]])
df = pd.DataFrame(rows, columns=["device", "label_index", "label_name", "channels", "median_spectrum", "std_spectrum"])

In [3]:
# add Tivita measurements:


def agg_cc_spectra_Tivita(cc_name: Path):
    df_Tiv = pd.DataFrame()
    cc_dir = settings.data_dirs["HeiPorSPECTRAL"] / "extra_technical_validation/Tivita_0202-00118" / cc_name
    for img_dir in list(cc_dir.iterdir()):
        path = DataPath(img_dir)
        median_table = path.read_colorchecker_mask()["median_table"]
        df_Tiv = pd.concat([df_Tiv, median_table])

    column_name = "median_normalized_spectrum" if normalization == 1 else "median_spectrum"
    df_Tiv = df_Tiv.groupby(["label_index", "label_name"], as_index=False)[column_name].agg(
        {
            "median_spectrum": lambda rows: np.median(np.stack(rows), axis=0),
            "std_spectrum": lambda rows: np.std(np.stack(rows), axis=0),
        }
    )

    df_Tiv["device"] = f"Tivita_{cc_name[:10]}"
    df_Tiv["channels"] = [np.linspace(500, 1000, 100)] * 24

    return df_Tiv


df_Tiv1 = agg_cc_spectra_Tivita("2022_12_25_colorchecker")
df_Tiv2 = agg_cc_spectra_Tivita("2022_12_30_colorchecker")
df = pd.concat([df, df_Tiv1, df_Tiv2])

In [4]:
df

,device,label_index,label_name,channels,median_spectrum,std_spectrum
0,spectrometer,1,dark_skin,"[500.311, 500.768, 501.225, 501.683, 502.14, 5...","[0.004816846270520448, 0.004824970004092594, 0...","[5.655524782996474e-05, 5.184966223713776e-05,..."
1,spectrometer,2,light_skin,"[500.311, 500.768, 501.225, 501.683, 502.14, 5...","[0.005594650864000915, 0.0056169832923717005, ...","[5.4545849505329166e-05, 5.784111114736063e-05..."
2,spectrometer,3,blue_sky,"[500.311, 500.768, 501.225, 501.683, 502.14, 5...","[0.009274281181750301, 0.009248072086987935, 0...","[4.9572159741060256e-05, 5.715101416433743e-05..."
3,spectrometer,4,foliage,"[500.311, 500.768, 501.225, 501.683, 502.14, 5...","[0.006673757486007695, 0.00673766866736257, 0....","[5.6967558556650644e-05, 6.58842979801498e-05,..."
4,spectrometer,5,blue_flower,"[500.311, 500.768, 501.225, 501.683, 502.14, 5...","[0.0055435173432195756, 0.005539052260874974, ...","[5.224041341081829e-05, 4.3628770254951783e-05..."
...,...,...,...,...,...,...
19,Tivita_2022_12_30,20,neutral_8,"[500.0, 505.050505050505, 510.1010101010101, 5...","[0.009680036, 0.009904543, 0.010149032, 0.0103...","[0.00013708066, 9.572458e-05, 6.1671526e-05, 3..."
20,Tivita_2022_12_30,21,neutral_65,"[500.0, 505.050505050505, 510.1010101010101, 5...","[0.010260498, 0.010419976, 0.010636309, 0.0107...","[0.00012707541, 8.4487496e-05, 5.7123874e-05, ..."
21,Tivita_2022_12_30,22,neutral_5,"[500.0, 505.050505050505, 510.1010101010101, 5...","[0.010492876, 0.010466981, 0.010654908, 0.0107...","[0.00012392685, 8.179253e-05, 4.7291844e-05, 3..."
22,Tivita_2022_12_30,23,neutral_35,"[500.0, 505.050505050505, 510.1010101010101, 5...","[0.010782469, 0.010351444, 0.010313116, 0.0102...","[0.00012619006, 6.857628e-05, 5.3044896e-05, 2..."


In [5]:
# Image of the colorchecker encoded as base64
colorchecker_img = (
    "<img width='850'"
    " src='...'/>"
)

In [6]:
fig = make_subplots(
    rows=6,
    cols=4,
    shared_xaxes=True,
    shared_yaxes=True,
    subplot_titles=list(ColorcheckerReader.label_colors_classic.keys()),
    horizontal_spacing=0.02,
    vertical_spacing=0.04,
)

linecolor = {"spectrometer": "rgb(136, 34, 85)", "Tivita_2022_12_25": "#2CA02C", "Tivita_2022_12_30": "#BCBD22"}

used_labels = []
for df_row in df.iterrows():
    label = df_row[1].device
    if label not in used_labels:
        used_labels.append(label)
        showlegend = True
    else:
        showlegend = False
    fig = add_std_fill(
        fig,
        df_row[1].median_spectrum,
        df_row[1].std_spectrum,
        x=df_row[1].channels,
        linecolor=linecolor[label],
        label=label,
        row=(df_row[1].label_index - 1) // 4 + 1,
        col=(df_row[1].label_index - 1) % 4 + 1,
        showlegend=showlegend,
    )

yaxis_title = "L1-normalized<br>reflectance [a.u.]" if normalization == 1 else "reflectance [a.u.]"
fig = colorchecker_fig_styling(fig, yaxis_title=yaxis_title)
if normalization is None:
    fig.update_yaxes(dtick=1)
    file_prefix = "Figure_08"
    file_suffix = "orig"
    caption_prefix = (
        "<strong>Figure 8 | A checkerboard-based technical validation of the original reflectance values prior to"
        " L1-normalization shows a relevant difference to other spectrometer measurements.</strong>"
    )
else:
    file_prefix = "Figure_07"
    file_suffix = "L1"
    caption_prefix = (
        "<strong>Figure 7 | A checkerboard-based technical validation confirms very good reproducibility and high"
        " validity of the HSI images.</strong>"
    )

target_dir = settings.results_dir / "open_data"
target_dir.mkdir(parents=True, exist_ok=True)
fig.write_image(target_dir / f"{file_prefix}_colour_checker_{file_suffix}.pdf", scale=1.1)
fig.update_xaxes(tickmode="auto")

html = add_figcaption(
    colorchecker_img + fig.to_html(full_html=False, include_plotlyjs=True, div_id="technical_validation"),
    title="Checkerboard-based technical validation",
    caption=(
        f"{caption_prefix} <strong>a</strong>, Reconstructed RGB image from a hyperspectral imaging (HSI) recording of"
        " a colorchecker board. <strong>b</strong>, Spectrometer (purple) and HSI (light and dark green) recordings"
        " for the 24 different color chips. The median spectrum across all available recordings (100 recordings for"
        " spectrometer, 13 recordings for HSI) is denoted by a solid line, the corresponding standard deviation as a"
        " shaded area."
    ),
)
compress_html(target_dir / f"{file_prefix}_colour_checker_{file_suffix}.html", html)

In [7]:
df_agg = df.groupby("device")["std_spectrum"].agg(lambda x: np.mean(np.stack(x))).reset_index()
df_agg

,device,std_spectrum
0,Tivita_2022_12_25,0.000041
1,Tivita_2022_12_30,0.000029
2,spectrometer,0.000141
